In [ ]:
!export PATH=/usr/local/cuda/bin:$PATH
!export CPATH=/usr/local/cuda/include:$CPATH
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH
!pip install --user torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install --user torch-scatter==latest+cu100 torch-sparse==latest+cu100 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html
!pip install --user torch-cluster==1.5.4+cu100 torch-spline-conv==1.1.1 torch-geometric==1.4.2 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html
!pip install --user ipympl pyntcloud

# !curl -sL https://deb.nodesource.com/setup_13.x | sudo -E bash -
# !sudo apt-get install -y nodejs

# !pip install  --user ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !sudo jupyter labextension install @jupyter-widgets/jupyterlab-manager

# Warning! Sometimes you have to use --no-cache-dir to avoid undefined symbol error

# Matplotlib
# !pip install —user ipympl

# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter labextension install jupyter-matplotlib

In [ ]:
# insrtall openfoam
!sudo apt-get install -y openssh-client
!sudo apt-get install -y software-properties-common
!sudo sh -c "wget -O - https://dl.openfoam.org/gpg.key | apt-key add -"
!sudo add-apt-repository http://dl.openfoam.org/ubuntu
!sudo apt-get update
!sudo apt-get -y install openfoam5

!pip install --user numpy-stl
!echo "export PATH=/cvlabdata2/home/artem/autofoam/bin:$PATH" >> ~/.bashrc 
!echo ". /opt/openfoam5/etc/bashrc" >> ~/.bashrc 

In [ ]:
!sudo apt update
!sudo wget https://github.com/mmatl/travis_debs/raw/master/xenial/mesa_18.3.3-0.deb
!sudo dpkg -i ./mesa_18.3.3-0.deb || true
!sudo apt install -f
!git clone https://github.com/mmatl/pyopengl.git
!pip install --user ./pyopengl

!pip install --user pyrender
!pip3 install --user mesh-to-sdf
!export PYOPENGL_PLATFORM=osmesa

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os
import sys
import time
import numpy as np
import json
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt 
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import (NNConv, GMMConv, GraphConv, Set2Set)
from torch_geometric.nn import (SplineConv, graclus, max_pool, max_pool_x, global_mean_pool)

#from neuralnet_pytorch.metrics import chamfer_loss

import trimesh

from visualization_utils import plot_mesh_3d

from models import *
from datasets import compute_lift, CDFDataset, CDFDatasetInMemory, compute_lift_faces, make_data_instance_from_stl
from visualization_utils import saveMeshPly
from sklearn.neighbors import KDTree

In [ ]:
m = trimesh.load('Expirements/OptimizationPaper/DeepSDFDrag/meshes/00025.ply')\
           .export('Expirements/Simulations/RawSimulations/inputs/mesh3DeepSDF25conservative_input.stl')

# Compute Statistics for Normalization

In [ ]:
objects = list()
for (dirpath, dirnames, filenames) in os.walk('/cvlabdata2/home/artem/Data/cars_refined/simulated/fld'):
    objects += [os.path.join(dirpath, file) for file in filenames if file[-4:] == '.fld']

mean_values = np.zeros((len(objects), 4))
std_values = np.zeros((len(objects), 4))

for idx, fld_path in tqdm(enumerate(objects), total=len(objects)):
    fld = np.genfromtxt(fld_path, delimiter=',', skip_header=1)
    fld[fld > 10e5] = np.nan
    fld = fld[~np.isnan(fld).any(axis=1)]

    answers = fld[:, 3:]

    for f in range(answers.shape[1]):
        mean_values[idx, f] = np.mean(answers[:, f])
        std_values[idx, f] = np.std(answers[:, f])
        
print(np.mean(mean_values, axis=0))
print(np.mean(std_values, axis=0))

In [ ]:
!pwd

In [ ]:
objects = list()
for (dirpath, dirnames, filenames) in os.walk('/cvlabdata2/home/artem/Data/cars_refined/simulated/scr'):
    objects += [os.path.join(dirpath, file) for file in filenames if file[-5:] == '.json' and file[0] != '.']

scr_data_store = np.zeros((len(objects), 12))

for idx, scr_path in tqdm(enumerate(objects), total=len(objects)):
    with open(scr_path) as scr_file:
        scr_data = json.load(scr_file)
        scr_data_store[idx, :3] = scr_data['pressure_drag']
        scr_data_store[idx, 3:6] = scr_data['viscous_drag']
        scr_data_store[idx, 6:9] = scr_data['pressure_moment']
        scr_data_store[idx, 9:] = scr_data['viscous_moment']  
        
print(np.mean(scr_data_store, axis=0))
print(np.std(scr_data_store, axis=0))

# Analyse the prediction of a trained Network

In [ ]:
def getR2Score(y, pred):
    mean_data = np.mean(y.numpy())
    sstot = np.sum((y.numpy() - mean_data) ** 2)
    ssreg = np.sum((pred.numpy() - mean_data) ** 2)
    ssres = np.sum((pred.numpy() - y.numpy()) ** 2)
    return 1 - ssres / sstot

def getModelReport(model, data_path="/cvlabdata2/home/artem/Data/cars_refined/simulated", 
                   data_step=1, global_features=False):
    val_dataset = CDFDatasetInMemory(data_path, train=False)
    val_loader = torch_geometric.data.DataLoader(val_dataset, batch_size=1, shuffle=False)
    
    
    device = "cuda:0"
    model = model.to(device)
    model.eval()

    r2_scores = []
    pathes = []
    mses = []
    verticies = []
    for batch in tqdm(val_loader):
        batch = batch.to(device)
        if global_features:
            local_preds, global_preds = model(batch.clone())
        else:
            local_preds = model(batch.clone())
    
        local_preds = local_preds.cpu().detach()
        data_instance = batch.y.cpu().detach()
            
        r2_scores.append([getR2Score(data_instance[:, 0], local_preds[:, 0]),
                          getR2Score(data_instance[:, 1], local_preds[:, 1]),
                          getR2Score(data_instance[:, 2], local_preds[:, 2]),
                          getR2Score(data_instance[:, 3], local_preds[:, 3]),
                          getR2Score(data_instance, local_preds)])
                         
        mses.append([F.mse_loss(local_preds[:, 0], data_instance[:, 0]).numpy(),
                     F.mse_loss(local_preds[:, 1], data_instance[:, 1]).numpy(),
                     F.mse_loss(local_preds[:, 2], data_instance[:, 2]).numpy(),
                     F.mse_loss(local_preds[:, 3], data_instance[:, 3]).numpy(),
                     F.mse_loss(local_preds, data_instance).numpy()])
        pathes.append(batch.path)
        verticies.append(data_instance.shape[0])
        
    r2_scores_ret, mses_ret = np.array(r2_scores), np.array(mses)   
    r2_scores, mses = r2_scores_ret, mses_ret 
    #r2_scores, mses = r2_scores_ret[r2_scores_ret[:, 0] > 0 , :], mses_ret[r2_scores_ret[:, 0] > 0, :]   
    
    print("Average number of verticies : ", int(np.mean(verticies)) )
    print("Number of Model Parameters  : ", sum(p.numel() for p in model.parameters()))
    print()
    print("                     |        MSE       |       R2    ")
    print("---------------------------------------------------------")
    print("Pressure             | %.4f +- %.4f | %.4f +- %.4f" % 
          (np.mean(mses[:, 0]), np.std(mses[:, 0]), np.mean(r2_scores[:, 0]), np.std(r2_scores[:, 0]) ) )
    print("Kinetic Energy       | %.4f +- %.4f | %.4f +- %.4f" % 
          (np.mean(mses[:, 1]), np.std(mses[:, 1]), np.mean(r2_scores[:, 1]), np.std(r2_scores[:, 1]) ))
    print("Omega                | %.4f +- %.4f | %.4f +- %.4f" % 
          (np.mean(mses[:, 2]), np.std(mses[:, 2]), np.mean(r2_scores[:, 2]), np.std(r2_scores[:, 2]) ))
    print("Turbulent Viscosity  | %.4f +- %.4f | %.4f +- %.4f" % 
          (np.mean(mses[:, 3]), np.std(mses[:, 3]), np.mean(r2_scores[:, 3]), np.std(r2_scores[:, 3]) ))
    print("Total                | %.4f +- %.4f | %.4f +- %.4f" % 
          (np.mean(mses[:, 4]), np.std(mses[:, 4]), np.mean(r2_scores[:, 4]), np.std(r2_scores[:, 4]) ))
    
    return r2_scores_ret, mses_ret, pathes



def getLiftModelReport(model, data_path="/cvlabdata2/home/artem/Data/cars_refined/simulated", 
                       data_step=1, global_features=False):
    val_dataset = CDFDatasetInMemory(data_path, connectivity=10, train=False, data_step=data_step)
    val_loader = torch_geometric.data.DataLoader(val_dataset, batch_size=1, shuffle=False)
    
    device = "cuda:0"
    model = model.to(device)

    predicted = []
    correct = []
    for batch in tqdm(val_loader):
        batch = batch.to(device)
        if global_features:
            local_preds, global_preds = model(batch.clone())
        else:
            local_preds = model(batch.clone())
        predicted.append(compute_lift_faces(batch, local_preds))
        correct.append(compute_lift_faces(batch, batch.y))
        
    print("R2 score : ", getR2Score(torch.tensor(predicted), torch.tensor(correct)))
        
    return predicted, correct

In [ ]:
model = SplineCNN4Pooling(3)
model.load_state_dict(torch.load("Expirements/SplineCNN4Pooling.nn"))
r2scores, mses, pathes = getModelReport(model, global_features=True)

In [ ]:
model = SplineCNN2(3)
model.load_state_dict(torch.load("Expirements/Spline2CDF_sparse.nn"))
r2scores, mses, pathes = getModelReport(model, data_step=100)

In [ ]:
model = SplineCNN2(3)
model.load_state_dict(torch.load("Expirements/Spline2CDF_sparse10.nn"))
r2scores, mses, pathes = getModelReport(model, data_step=10)

In [ ]:
model = SplineCNN2(3)
model.load_state_dict(torch.load("Expirements/Spline2CDF.nn"))
r2scores, mses, pathes = getModelReport(model, data_step=1)

In [ ]:
model = SplineCNN4(3)
model.load_state_dict(torch.load("Expirements/Spline4CDF_sparse.nn"))
r2scores4, mses4, pathes4 = getModelReport(model, data_step=100)

In [ ]:
model = SplineCNN4(3)
model.load_state_dict(torch.load("Expirements/Spline4CDF_sparse10.nn"))
r2scores, mses, pathes = getModelReport(model, data_step=10)

In [ ]:
model = SplineCNN4(3)
model.load_state_dict(torch.load("Expirements/Spline4CDF.nn"))
r2scores, mses, pathes = getModelReport(model)

In [ ]:
model = SplineCNN2(3)
model.load_state_dict(torch.load("Expirements/Spline2CDF.nn"))
predicted, correct = getLiftModelReport(model, data_step=1)

In [ ]:
model = SplineCNN2Pooling(3)
model.load_state_dict(torch.load("Expirements/SplineCNN2PoolingNe.nn"))
r2_scores, msest, pathes = getModelReport(model, global_features=True)

In [ ]:
for p, score in zip(pathes, r2_scores[:, 0]):
    if score < 0:
        print(score, p)

In [ ]:
model = SplineCNN8Residuals(3)

model.load_state_dict(torch.load("Expirements/SplineCNN8BatchNorm.nn"))
r2_scores, msest, pathes = getModelReport(model, global_features=False)

In [ ]:
for p, score in zip(pathes, r2_scores[:, 0]):
    if score < 0:
        print(score, p)
        
print('=====')

for p, score in zip(pathes, r2_scores[:, 0]):
    if score >= 0:
        print(score, p)

In [ ]:
nice_shapes = [x for y,x in sorted(zip(r2scores[:, 0], range(len(r2scores)))) if y > 0]
getR2Score(torch.tensor(predicted)[nice_shapes], torch.tensor(correct)[nice_shapes])

### Analyse worst meshes

In [ ]:
bad_shapes = [(y, x[0]) for y,x in sorted(zip(r2scores[:, 0], pathes)) if y < 0]
for score, path in bad_shapes:
    print(path, ' : ', score)

In [ ]:
for score, path in bad_shapes:
    data_instance = make_data_instance_from_fld(path)
    saveMeshPly(data_instance.x, data_instance.y, 'Expirements/BadTestShapes/%s%.4f.ply' % (path.split('/')[-1][:4], score) )

In [ ]:
model = SplineCNN2(3)
model.load_state_dict(torch.load("Expirements/Spline2CDF.nn"))
device = "cuda:0"
model = model.to(device)

for score, path in bad_shapes:
    data_instance = make_data_instance_from_fld(path)
    data_instance = data_instance.to(device)
    prediction = model(data_instance)
    saveMeshPly(data_instance.x.cpu().detach(), prediction.cpu().detach(), 'Expirements/BadTestShapes/Prediction%s%.4f.ply' % 
                (path.split('/')[-1][:4], score) )

## Save mesh and get Global results

In [ ]:
from visualization_utils import saveMeshPly

data_instance= make_data_instance_from_fld(
                    '/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/fld/0517_0005.fld')
model = SplineCNN4Pooling(3)
model.load_state_dict(torch.load("Expirements/SplineCNN4Pooling.nn"))

device = "cuda:0"
model = model.to(device)
loader = torch_geometric.data.DataLoader([data_instance], batch_size=1, shuffle=False)
batch = next(iter(loader)).to(device)

In [ ]:
local_preds, global_preds = model(batch.clone())
local_preds, global_preds = local_preds.cpu().detach(), global_preds.cpu().detach()

In [ ]:
local_preds = model(batch.clone())
local_preds = local_preds.cpu().detach()

In [ ]:
stacked_answers = list(data_instance.pressure_drag[0].numpy()) + list(data_instance.viscous_drag[0].numpy()) +\
                  list(data_instance.pressure_moment[0].numpy()) + list(data_instance.viscous_moment[0].numpy())

print("Predicted : ", global_preds[0].numpy()[:3])
print("            ", global_preds[0].numpy()[3:6])
print("            ", global_preds[0].numpy()[6:9])
print("            ", global_preds[0].numpy()[9:])
print("GT        : ", stacked_answers[:3])
print("            ", stacked_answers[3:6])
print("            ", stacked_answers[6:9])
print("            ", stacked_answers[9:])
print()
print("Pressure drag   : ", F.mse_loss(global_preds[:, :3], data_instance.pressure_drag).numpy())
print("Viscous drag    : ", F.mse_loss(global_preds[:, 3:6], data_instance.viscous_drag).numpy())
print("Pressure moment : ", F.mse_loss(global_preds[:, 6:9], data_instance.pressure_moment).numpy())
print("Viscous moment  : ", F.mse_loss(global_preds[:, 9:], data_instance.viscous_moment).numpy())

In [ ]:
saveMeshPly(batch.x, batch.y, 'Expirements/BadTestShapes/517.ply')

## Make a mapping from old names to new ones

In [ ]:
def generateNmaesMapping(root):
    mapping = {}
    objects = []
    for (dirpath, dirnames, filenames) in os.walk(os.path.join(root, 'scr')):
        objects += [(os.path.join(dirpath, file), file[:-5]) for file in filenames if file[-5:] == '.json' and file[0] != '.']
    for path, name in objects:
        with open(path, 'r') as json_data:
             origId = json.load(json_data)['stl_id']
        mapping[origId] = name
    return mapping

In [ ]:
generateNmaesMapping('/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy')

# Old Vis

In [ ]:
colors=np.copy(data_instance.y[:, 0])
threshold = np.percentile(colors, 99.5)
colors[colors > threshold] = threshold

plt.hist(colors, bins=100)
plt.show()

In [ ]:
xadj = np.loadtxt('/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/xadj/0000.xadj')
fld = np.genfromtxt('/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/fld/0000_0005.fld', delimiter=',', skip_header=1)

fld_coords = sorted(fld[:, :3],key=lambda x: x[0])
xadj_coords = sorted(xadj[:, :3], key=lambda x: x[0])

In [ ]:
xadj.shape

In [ ]:
fld.shape

In [ ]:
xadj 

In [ ]:
from pyntcloud import PyntCloud
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def plot_points_from_fld(fld, step_pt=2):
    colors=np.copy(fld[:,3])
    threshold = np.percentile(colors, 99.5)
    colors[colors > threshold] = threshold
    norm = mpl.colors.Normalize(vmin=np.min(colors), vmax=np.max(colors))
    cmap = cm.hot
    m = cm.ScalarMappable(norm=norm, cmap=cmap)

    fig = plt.figure(figsize=(18, 10))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(-fld[::step_pt,2], fld[::step_pt,0], fld[::step_pt,1], s=5, c=m.to_rgba(colors[::step_pt]))
    plt.colorbar(m)
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    plt.show()

In [ ]:
plot_points_from_fld(fld)

In [ ]:
colors=np.copy(fld[:,3])
threshold = np.percentile(colors, 99.5)
colors[colors > threshold] = threshold

plt.hist(colors, bins=100)
plt.show()

In [ ]:
fld.shape

In [ ]:
from custom_utils import plot_points_from_torch

In [ ]:
plot_points_from_torch(np.array(sorted(fld, key=lambda x: x[0]))[:, :3])

In [ ]:
plot_points_from_torch(np.array(sorted(xadj, key=lambda x: x[0]))[:, :3])

In [ ]:
objects = list()
for (dirpath, dirnames, filenames) in os.walk("/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/fld/"):
    objects += [os.path.join(dirpath, file) for file in filenames if file[-4:] == '.fld']

for path in objects:
    a = np.genfromtxt(path, delimiter=',', skip_header=1)
    if a.max() > 1e10: 
        print(a.max(), path)

In [ ]:
a.argmax(axis=0)

In [ ]:
a[80938]

# Trimesh Visualization

In [ ]:
model = SplineCNN8Residuals(3).cuda()
model.load_state_dict(torch.load("Expirements/SplineCNN8BatchNorm.nn"))
#m = model.eval()

In [ ]:
data_instance = make_data_instance_from_stl('/cvlabdata2/home/artem/Data/cars_refined/simulated/fld/0617_0005.fld')
data_instance.to('cuda:0')
#data_instance.y = 0
prediction = model(data_instance)

In [ ]:
def createColoredTrimesh(data_instance, target):
    norm = mpl.colors.Normalize(vmin= -8, vmax=8)
    cmap = cm.hot
    m = cm.ScalarMappable(norm=norm, cmap=cmap)

    mesh = trimesh.Trimesh(vertices=data_instance.pos, 
                           faces=data_instance.face.t(),
                           vertex_colors=list(map(lambda c: m.to_rgba(c),  target)))
    
    return mesh
    

In [ ]:
mesh = createColoredTrimesh(data_instance.to('cpu'), data_instance.y[:, 0].cpu())
mesh.show()

In [ ]:
m = mesh.export("../Expirements/savedMeshes/622gt.ply")

In [ ]:
mesh = createColoredTrimesh(data_instance.to('cpu'), prediction[:, 0].cpu().detach())
mesh.show()

In [ ]:
m = mesh.export("../Expirements/savedMeshes/622pr_train.ply")

In [ ]:
plt.hist(data_instance.y[:, 0], bins=50)
plt.show()

In [ ]:
plt.hist(prediction[:, 0].cpu().detach(), bins=100)
plt.show()

In [ ]:
prediction[:, 0].mean()

# Check New Dataset

In [ ]:
train_dataset = CDFDatasetInMemory('/cvlabdata2/home/artem/Data/cars_refined/simulated')

In [ ]:
test_dataset = CDFDatasetInMemory('/cvlabdata2/home/artem/Data/cars_refined/simulated', train=False)

In [ ]:
test_loader = torch_geometric.data.DataLoader(test_dataset, batch_size=1, shuffle=False)